In [ ]:
from google.colab import drive #mounting drive to read and write data to drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#if running in Jupyter first we install scikit-surprise by going to anaconda comand prompt and typing the command below
#conda install -c conda-forge scikit-surprise
! pip install scikit-surprise
import surprise
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import os
print(os.listdir("/content/drive/MyDrive/books"))

['books.csv', 'to_read.csv', 'ratings.csv', 'book_tags.csv', 'tags.csv', 'sample_book.xml']


In [ ]:
raw=pd.read_csv('/content/drive/MyDrive/books/ratings.csv')
raw.drop_duplicates(inplace=True)
raw=raw.head(1000)
print('we have',raw.shape[0], 'ratings')
print('the number of unique users we have is:', len(raw.user_id.unique()))
print('the number of unique books we have is:', len(raw.book_id.unique()))
print("The median user rated %d books."%raw.user_id.value_counts().median())
print('The max rating is: %d'%raw.rating.max(),"the min rating is: %d"%raw.rating.min())
raw.head()

we have 1000 ratings
the number of unique users we have is: 288
the number of unique books we have is: 10
The median user rated 3 books.
The max rating is: 5 the min rating is: 1


,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [ ]:
ratings.describe

<bound method NDFrame.describe of      book_id  user_id  rating
0          1      314       5
1          1      439       3
2          1      588       5
3          1     1169       4
4          1     1185       4
..       ...      ...     ...
995       10    50999       5
996       10    51166       3
997       10    51838       4
998       10    52036       4
999       10    53292       5

[1000 rows x 3 columns]>

In [ ]:
rawTrain,rawholdout = train_test_split(ratings, test_size=0.25 )
# when importing from a DF, you only need to specify the scale of the ratings.
reader = surprise.Reader(rating_scale=(1,5)) 
#into surprise:
data = surprise.Dataset.load_from_df(rawTrain,reader)
holdout = surprise.Dataset.load_from_df(rawholdout,reader)
holdout

In [ ]:
kSplit = surprise.model_selection.split.KFold(n_splits=10, shuffle=True) # split data into folds.

In [ ]:
sim_options = sim_options = {'name': 'cosine','user_based': False  # compute  similarities between items
               } 
collabKNN = surprise.KNNBasic(k=40,sim_options=sim_options) #try removing sim_options. You'll find memory errors. 
rmseKNN = []
rmseSVD = []
rmseCo = []
rmseSlope = []
for trainset, testset in kSplit.split(data): #iterate through the folds.
    collabKNN.fit(trainset)
    predictionsKNN = collabKNN.test(testset)
    rmseKNN.append(surprise.accuracy.rmse(predictionsKNN,verbose=True))#get root means squared error

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0929
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9572
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0312
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9745
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0059
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0864
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.2768
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.1174
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0486
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0568


In [ ]:
funkSVD = surprise.prediction_algorithms.matrix_factorization.SVD(n_factors=30,n_epochs=10,biased=True)

In [ ]:
min_error = 1
for trainset, testset in kSplit.split(data): #iterate through the folds.
    funkSVD.fit(trainset)
    predictionsSVD = funkSVD.test(testset)
    rmseSVD.append(surprise.accuracy.rmse(predictionsSVD,verbose=True))#get root means squared error

RMSE: 1.1340
RMSE: 0.9008
RMSE: 0.9889
RMSE: 1.1420
RMSE: 0.9877
RMSE: 1.0492
RMSE: 1.1173
RMSE: 1.0545
RMSE: 1.0356
RMSE: 1.0808


In [ ]:
coClus = surprise.prediction_algorithms.co_clustering.CoClustering(n_cltr_u=4,n_cltr_i=4,n_epochs=25) 
for trainset, testset in kSplit.split(data): #iterate through the folds.
    coClus.fit(trainset)
    predictionsCoClus = coClus.test(testset)
    rmseCo.append(surprise.accuracy.rmse(predictionsCoClus,verbose=True))#get root means squared error

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


RMSE: 1.0218
RMSE: 1.1310
RMSE: 1.1448
RMSE: 1.1877
RMSE: 1.4609
RMSE: 1.2365
RMSE: 1.1138
RMSE: 1.0758
RMSE: 1.2221
RMSE: 1.1173


In [ ]:
slopeOne = surprise.prediction_algorithms.slope_one.SlopeOne()

In [ ]:
for trainset, testset in kSplit.split(data): #iterate through the folds.
    slopeOne.fit(trainset)
    predictionsSlope = slopeOne.test(testset)
    rmseSlope.append(surprise.accuracy.rmse(predictionsSlope,verbose=True))#get root means squared error

RMSE: 1.1184
RMSE: 1.0616
RMSE: 0.9774
RMSE: 1.1738
RMSE: 1.1191
RMSE: 1.0650
RMSE: 1.1312
RMSE: 1.1439
RMSE: 0.9495
RMSE: 1.0395


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [ ]:
from surprise import AlgoBase

class HybridModel(AlgoBase):

    def __init__(self, models, weights, sim_options={}):
        AlgoBase.__init__(self)
        self.models = models
        self.weights = weights

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        
        for model in self.models:
            model.fit(trainset)
                
        return self

    def estimate(self, user_id, item_id):
        
        scores_sum = 0
        weights_sum = 0
        
        for i in range(len(self.models)):
            scores_sum += self.models[i].estimate(user_id, item_id) * self.weights[i] # 3*1/4+4*3/4 laga ra
            weights_sum += self.weights[i] # always becomes one
            
        return scores_sum / weights_sum

In [ ]:
class HybridFacto(surprise.AlgoBase):
    def __init__(self,epochs, learning_rate,num_models):
        self.alpha = np.array([1/len(num_models)]*len(num_models))
        self.epochs = epochs
        self.learning_rate = learning_rate
    def fit(self,holdout):
        holdout=holdout.build_full_trainset().build_testset()
        for epoch in range(self.epochs):
          predictions = np.array([collabKNN.test(holdout),funkSVD.test(holdout),coClus.test(holdout),slopeOne.test(holdout)])
          maeGradient = [surprise.accuracy.mae(prediction) for prediction in predictions]
          newalpha = self.alpha - learning_rate * maeGradient  
            #convergence check:
          if newalpha - self.alpha < 0.001:
                break
          self.alpha = newalpha
    def estimate(self,u,i): 
          if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('User and/or item is unkown.')
            algoResults = np.array([collabKNN.predict(u,i),funkSVD.predict(u,i),coClus.predict(u,i),slopeOne.predict(u,i)])
            return np.sum(np.dot(self.alpha,algoResults))
        

In [ ]:
hybrid = HybridFacto(10,0.05,4)
hybrid.fit(holdout)
rmseHyb = []
for trainset, testset in kSplit.split(data): #iterate through the folds.
    predhybrid = hybrid.test(testset)
    rmseHyb.append(surprise.accuracy.rmse(predhybrid))